In [1]:
!pip install -U accelerate
!pip install -U transformers
!pip install --upgrade huggingface_hub
!pip install evaluate

/bin/bash: /home/gariman/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/gariman/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/gariman/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/gariman/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from google.colab import files

uploaded = files.upload()
fnames = []
for fn in uploaded.keys():
  fnames.append(fn)
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
print("nosql_injection_dataset.json")

In [2]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

0

In [5]:
import torch
import accelerate
from transformers import AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import json

# Define the dataset class
import torch
from torch.utils.data import Dataset, TensorDataset

import torch
from torch.utils.data import Dataset

class NoSQLInjectionDataset(Dataset):
    def __init__(self, dataset, tokenizer, split_ratio=0.9):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.split_ratio = split_ratio
        self.train_dataset = self.dataset[:int(len(self.dataset) * self.split_ratio)]
        self.eval_dataset = self.dataset[int(len(self.dataset) * self.split_ratio):]

    def __len__(self):
        return len(self.train_dataset)

    def __getitem__(self, idx):
        sample = self.train_dataset[idx]
        template = sample["template"]
        payload = sample["payload"]

        # Tokenize template and payload
        tokenized_inputs = self.tokenizer.encode_plus(
            template,
            payload,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )

        input_ids = tokenized_inputs["input_ids"].squeeze()
        attention_mask = tokenized_inputs["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids
        }

    def get_eval_dataset(self):
        eval_dataset = []
        for sample in self.eval_dataset:
            template = sample["template"]
            payload = sample["payload"]

            tokenized_inputs = self.tokenizer.encode_plus(
                template,
                payload,
                truncation=True,
                padding="max_length",
                max_length=512,
                return_tensors="pt"
            )

            input_ids = tokenized_inputs["input_ids"].squeeze()
            attention_mask = tokenized_inputs["attention_mask"].squeeze()

            eval_dataset.append({
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "labels": input_ids
            })

        return eval_dataset

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
# Load the tokenizer and model
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")
# Convert the dataset into a PyTorch Dataset
train_dataset = NoSQLInjectionDataset(dataset, tokenizer)
eval_dataset = train_dataset.get_eval_dataset()
# Set up the training arguments and trainer
training_args = TrainingArguments(
    output_dir="fine-tuned-distilroberta-nosql-injection",
    evaluation_strategy="epoch",
    num_train_epochs=150, 
    per_device_train_batch_size=8,
    save_steps=1000, 
    logging_steps=100, # used to adjust the frequency of logging loss values
    logging_dir="./logs",
    overwrite_output_dir=True,
    learning_rate=2e-5,  # Adjust as needed
    warmup_steps=100,  # Adjust as needed
    weight_decay=0.01,  # Adjust as needed
    push_to_hub=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model

# Push the fine-tuned model to the Hugging Face Model Hub
trainer.push_to_hub("fine-tuned-distilroberta-nosql-injection")
tokenizer.push_to_hub("fine-tuned-distilroberta-nosql-injection")
print("Fine-tuned model saved and pushed to the Hugging Face Model Hub.")

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/gariman/Downloads/angus/fine-tuned-distilroberta-nosql-injection is already a clone of https://huggingface.co/ankush-003/fine-tuned-distilroberta-nosql-injection. Make sure you pull the latest changes with `repo.git_pull()`.
/home/gariman/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of A

  0%|          | 0/11850 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.3011210858821869, 'eval_runtime': 1.3023, 'eval_samples_per_second': 53.753, 'eval_steps_per_second': 6.911, 'epoch': 1.0}
{'loss': 1.3847, 'learning_rate': 2e-05, 'epoch': 1.27}


...

{'eval_loss': 0.07726605981588364, 'eval_runtime': 1.3462, 'eval_samples_per_second': 51.999, 'eval_steps_per_second': 6.686, 'epoch': 79.0}
{'loss': 0.0119, 'learning_rate': 9.446808510638299e-06, 'epoch': 79.75}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.01794789358973503, 'eval_runtime': 1.34, 'eval_samples_per_second': 52.24, 'eval_steps_per_second': 6.717, 'epoch': 80.0}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.011820195242762566, 'eval_runtime': 1.3321, 'eval_samples_per_second': 52.549, 'eval_steps_per_second': 6.756, 'epoch': 81.0}
{'loss': 0.0205, 'learning_rate': 9.276595744680852e-06, 'epoch': 81.01}


In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
trainer.push_to_hub()
tokenizer.push_to_hub()

In [ ]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", "ankush-003/fine-tuned-distilroberta-nosql-injection")

mask_filler("{username: <mask>, password: anksh}", top_k=3)